In [15]:
import spatioloji_s as sj
import os
import numpy as pd
import pandas as pd
import psutil
import os

In [16]:
mem = psutil.virtual_memory()
print(f"Total memory available: {mem.total / 1024**3:.2f} GB")
cpus = os.cpu_count()
print(f"Total cpu cores available: {cpus}")
print(f"current work directory is {os.getcwd()}")

Total memory available: 251.40 GB
Total cpu cores available: 64
current work directory is /carc/scratch/projects/amitra2016502/spatioloji_s/example


In [17]:
# prepare folder paths
data_read = '../../CosMx/OV/' # for raw data
data_save = '../../CosMx/OV/' # for processed data or intermediate result
analysis_save = '../../CosMx/OV/' # for plots
image_dir = '../../CosMx/OV/CellComposite/'

In [18]:
# Load expression matrix and cell metadata
expr_matrix = pd.read_csv(data_read+'Run5452_S2_exprMat_file.csv')
cell_metadata = pd.read_csv(data_read+'Run5452_S2_metadata_file.csv')


In [20]:
expr_matrix
# we have NegPrb, gene symbols, fov ids and cell ids as columns

,fov,cell_ID,AATK,ABL1,ABL2,ACE,ACE2,ACKR1,ACKR3,ACKR4,...,NegPrb13,NegPrb14,NegPrb15,NegPrb16,NegPrb18,NegPrb19,NegPrb20,NegPrb21,NegPrb22,NegPrb23
0,1,0,50,76,77,62,82,42,44,68,...,40,51,42,80,44,46,74,77,77,69
1,1,1,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,1,2,0,0,0,0,0,0,0,0,...,0,1,0,0,0,0,0,0,0,0
3,1,3,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,1,4,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
39954,22,3537,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
39955,22,3538,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
39956,22,3539,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,1,0
39957,22,3540,0,0,0,0,0,0,0,0,...,0,0,0,0,1,0,0,0,0,0


In [21]:
cell_metadata
# we have fov ids, cell ids, local X/Y coordinates and global X/Y coordiantes

,fov,cell_ID,Area,AspectRatio,CenterX_local_px,CenterY_local_px,CenterX_global_px,CenterY_global_px,Width,Height,Mean.MembraneStain,Max.MembraneStain,Mean.PanCK,Max.PanCK,Mean.CD45,Max.CD45,Mean.CD3,Max.CD3,Mean.DAPI,Max.DAPI
0,1,1,9074,1.97,1757,3612,5501.444444,10784.222222,144,73,21,119,77,1686,21,803,9,59,811,1835
1,1,2,7494,1.55,4931,3610,8675.444444,10782.222222,119,77,4,129,380,2126,10,830,1,27,153,1098
2,1,3,6621,1.25,5343,3609,9087.444444,10781.222222,99,79,2,97,15,1278,9,776,0,25,426,1486
3,1,4,3451,1.09,250,3617,3994.444444,10789.222222,70,64,1,104,276,1858,19,1040,1,73,11,143
4,1,5,10484,1.23,2758,3596,6502.444444,10768.222222,129,105,17,134,35,1657,18,984,4,47,395,1374
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
39934,22,3537,1774,1.73,4045,21,2972.777778,-9101.222222,71,41,68,118,3239,3878,17,102,17,56,644,929
39935,22,3538,2871,1.79,4180,24,3107.777778,-9098.222222,84,47,79,154,3046,3878,27,86,16,46,818,1237
39936,22,3539,2964,1.56,734,25,-338.222222,-9097.222222,75,48,5,50,888,3878,2,148,2,36,150,666
39937,22,3540,2968,2.14,4392,22,3319.777778,-9100.222222,90,42,75,164,3062,3878,12,67,13,53,420,756


In [22]:
# Create unique cell identifiers by combining FOV and cell ID
expr_matrix['fov'] = expr_matrix['fov'].astype(str)
expr_matrix['cell'] = expr_matrix['fov'].astype(str)+'_'+expr_matrix['cell_ID'].astype(str)
expr_matrix.index = expr_matrix.cell
cell_metadata['fov'] = cell_metadata['fov'].astype(str)
cell_metadata['cell'] = cell_metadata['fov'].astype(str)+'_'+cell_metadata['cell_ID'].astype(str)
cell_metadata.index = cell_metadata.cell

In [23]:
cell_metadata

,fov,cell_ID,Area,AspectRatio,CenterX_local_px,CenterY_local_px,CenterX_global_px,CenterY_global_px,Width,Height,...,Max.MembraneStain,Mean.PanCK,Max.PanCK,Mean.CD45,Max.CD45,Mean.CD3,Max.CD3,Mean.DAPI,Max.DAPI,cell
cell,,,,,,,,,,,,,,,,,,,,,
1_1,1,1,9074,1.97,1757,3612,5501.444444,10784.222222,144,73,...,119,77,1686,21,803,9,59,811,1835,1_1
1_2,1,2,7494,1.55,4931,3610,8675.444444,10782.222222,119,77,...,129,380,2126,10,830,1,27,153,1098,1_2
1_3,1,3,6621,1.25,5343,3609,9087.444444,10781.222222,99,79,...,97,15,1278,9,776,0,25,426,1486,1_3
1_4,1,4,3451,1.09,250,3617,3994.444444,10789.222222,70,64,...,104,276,1858,19,1040,1,73,11,143,1_4
1_5,1,5,10484,1.23,2758,3596,6502.444444,10768.222222,129,105,...,134,35,1657,18,984,4,47,395,1374,1_5
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
22_3537,22,3537,1774,1.73,4045,21,2972.777778,-9101.222222,71,41,...,118,3239,3878,17,102,17,56,644,929,22_3537
22_3538,22,3538,2871,1.79,4180,24,3107.777778,-9098.222222,84,47,...,154,3046,3878,27,86,16,46,818,1237,22_3538
22_3539,22,3539,2964,1.56,734,25,-338.222222,-9097.222222,75,48,...,50,888,3878,2,148,2,36,150,666,22_3539


In [24]:
expr_matrix

,fov,cell_ID,AATK,ABL1,ABL2,ACE,ACE2,ACKR1,ACKR3,ACKR4,...,NegPrb14,NegPrb15,NegPrb16,NegPrb18,NegPrb19,NegPrb20,NegPrb21,NegPrb22,NegPrb23,cell
cell,,,,,,,,,,,,,,,,,,,,,
1_0,1,0,50,76,77,62,82,42,44,68,...,51,42,80,44,46,74,77,77,69,1_0
1_1,1,1,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,1_1
1_2,1,2,0,0,0,0,0,0,0,0,...,1,0,0,0,0,0,0,0,0,1_2
1_3,1,3,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,1_3
1_4,1,4,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,1_4
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
22_3537,22,3537,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,22_3537
22_3538,22,3538,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,22_3538
22_3539,22,3539,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,1,0,22_3539


In [25]:
# Filter to keep only cells that exist in both expression and metadata
common_cells = expr_matrix.index.intersection(cell_metadata.index)
expr_matrix = expr_matrix.loc[common_cells]
cell_metadata = cell_metadata.loc[common_cells]

In [27]:
expr_matrix.index.tolist() == cell_metadata.index.tolist()

True

In [28]:
# Load spatial information (FOV positions and cell polygons)
fov_positions_orig = pd.read_csv(data_read+'Run5452_S2_fov_positions_file.csv')
fov_positions_orig['fov'] = fov_positions_orig['fov'].astype(str)
polygon_file_orig = pd.read_csv(data_read+'Run5452_S2-polygons.csv')
polygon_file_orig['fov'] = polygon_file_orig['fov'].astype(str)
polygon_file_orig['cell'] = polygon_file_orig['fov'].astype(str)+'_'+polygon_file_orig['cell_ID'].astype(str)

In [6]:
sj.data.spatioloji()

TypeError: spatioloji.__init__() missing 4 required positional arguments: 'polygons', 'cell_meta', 'adata', and 'fov_positions'

Total memory available: 251.40 GB
Total cpu cores available: 64
current work directory is /carc/scratch/projects/amitra2016502/spatioloji_s/example


In [14]:

os.listdir("../../CosMx/OV/CellComposite")

['CellComposite_F001.jpg',
 'CellComposite_F015.jpg',
 'CellComposite_F004.jpg',
 'CellComposite_F017.jpg',
 'CellComposite_F009.jpg',
 'CellComposite_F013.jpg',
 'CellComposite_F014.jpg',
 'CellComposite_F008.jpg',
 'CellComposite_F012.jpg',
 'CellComposite_F002.jpg',
 'CellComposite_F018.jpg',
 'CellComposite_F007.jpg',
 'CellComposite_F005.jpg',
 'CellComposite_F021.jpg',
 'CellComposite_F022.jpg',
 'CellComposite_F011.jpg',
 'CellComposite_F010.jpg',
 'CellComposite_F016.jpg',
 'CellComposite_F019.jpg',
 'CellComposite_F006.jpg']